# Ban User 목록 불러오기

In [ ]:
import requests

# Banned Player List 가져오기
url = "https://api.gametools.network/"
response = requests.get(url+"bfban/banned_players/")

banned_user_ids = []
if response.status_code == 200:
    # 성공적으로 데이터를 받아옴
    data = response.json() # JSON 응답을 파이썬 딕셔너리로 변환
    banned_user_ids = data["personaids"]
else:
    # 오류 처리
    print(f'Error: {response.status_code}')

 # Banned Player 닉네임 등 수집

## 데이터 수집하기

In [ ]:
saved_idx = 99999

In [ ]:
from selenium.common import InvalidSessionIdException
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By

import pandas as pd
import time

# 데이터를 저장할 리스트 초기화
data = []

# Chrome 바이너리 위치와 ChromeDriver 경로 설정
chrome_binary_path = "E:/Github/BattleField-Bot-Detection/dataset/src/chrome/chrome.exe"  # Chrome 실행 파일 위치
chromedriver_path = "E:/Github/BattleField-Bot-Detection/dataset/src/chrome/chromedriver.exe"  # ChromeDriver 경로

# Selenium 드라이버 설정
options = webdriver.ChromeOptions()
options.binary_location = chrome_binary_path  # Chrome 바이너리 위치 지정

try:
    driver = webdriver.Chrome(service=Service(chromedriver_path), options=options)  # or whichever driver you're using
except InvalidSessionIdException:
    print("Session became invalid. Restarting...")
    driver = webdriver.Chrome(service=Service(chromedriver_path), options=options)  # Restart the driver
    # Continue from where you left off or restart your tasks

# 각 사용자 ID에 대해 반복
for idx in range(saved_idx, len(banned_user_ids)):
    try:
        # 사용자 ID에 대한 URL 생성
        driver.get(f'https://bfban.com/player/{banned_user_ids[idx]}')
        
        # 페이지 로딩 대기
        if idx % 1000 == 0:
            time.sleep(10)
        else:
            time.sleep(3)
        
        user_type = driver.find_element(By.CLASS_NAME, "ivu-tag-text.ivu-tag-color-white").text
        
        game_img_src = driver.find_element(By.CSS_SELECTOR, ".ivu-tag-text.ivu-tag-color-default img").get_attribute('src')
        
        # 여러 <span> 요소들 중 최대 3개까지 추출
        bot_types = driver.find_elements(By.CSS_SELECTOR, ".ivu-tag.ivu-tag-size-default.ivu-tag-warning.ivu-tag-checked > span")
        bot_type_texts = [element.text for element in bot_types[:3]]  # 최대 3개의 요소 텍스트 추출
        
        date_times = []
        datetime_element = driver.find_elements(By.CSS_SELECTOR, "span.ivu-time")
        for element in datetime_element:
            if element.text == "":
                continue
            elif element.text != "":
                date_times.append(element.text)
            else:
                pass
        Firstly_reported_date = date_times[0]
        Recently_reported_date = date_times[1]
        
        # 추출된 텍스트들을 데이터에 추가
        data_entry = {
            'Type': user_type,
            'ID': banned_user_ids[idx],
            'game_img_src': game_img_src,
            'Firstly reported date': Firstly_reported_date,
            'Recently reported date': Recently_reported_date
        }
        
        # 동적으로 키 생성하여 추출된 각 bot_type 저장
        for i, bot_type_text in enumerate(bot_type_texts):
            data_entry[f'bot_type_{i}'] = bot_type_text
        
        data.append(data_entry)
        
        # 데이터프레임 생성 및 CSV 파일로 저장
        df = pd.DataFrame(data)
        csv_file_path = './band_user_data_selenium.csv'
        if saved_idx == 0:
            df.to_csv(csv_file_path, index=False, mode='w')
        else:
            df.to_csv(csv_file_path, index=False, mode='a', header=False)
        saved_idx = idx + 1
        print(f"데이터 저장 중: {saved_idx} / {len(banned_user_ids)}", end='\r')
        
        
        
    except Exception as e:
        # 드라이버 종료
        driver.quit()
        # 새로운 드라이버 생성
        try:
            driver = webdriver.Chrome(service=Service(chromedriver_path), options=options)  # or whichever driver you're using
        except InvalidSessionIdException:
            print("Session became invalid. Restarting...")
            driver = webdriver.Chrome(service=Service(chromedriver_path), options=options)  # Restart the driver
            # Continue from where you left off or restart your tasks

            print(f"Error retrieving data for user ID {banned_user_ids[idx]}: {e}")
            user_type = game_img_src = Firstly_reported_date = Recently_reported_date = ''
            bot_type_texts = []
        
# 드라이버 종료
driver.quit()

print(f'CSV 파일이 생성되었습니다: {csv_file_path}')

## 데이터 중복 제거하기

### 잘못된 데이터 인덱스 수정하기

In [ ]:
import dask.dataframe as dd
import pandas as pd

# Dask DataFrame from CSV
ddf = dd.read_csv('./band_user_data_selenium - Copy.csv', low_memory=False, on_bad_lines='warn', dtype={'ID': 'object'})

# Persist the DataFrame in memory
ddf = ddf.persist()

# Repartition Dask DataFrame into 10 partitions
ddf = ddf.repartition(npartitions=10)

# Required column names
new_index = ["Type", "ID", "game_img_src", "Firstly reported date", "Recently reported date", "bot_type_0", "bot_type_1", "bot_type_2"]

# Reindex within each partition and convert to Pandas DataFrame
ddf = ddf.map_partitions(lambda df: df.reindex(columns=new_index, fill_value=pd.NA))

# Compute the final DataFrame
df = ddf.compute()

# Print results
print(df.head())


In [ ]:
df.to_csv('./band_user_data_selenium - Copy1.csv', index=False)

### 데이터 중복 제거하기

In [ ]:
# pandas를 사용하여 중복 제거
import dask.dataframe as dd

# CSV 파일 로드
df = dd.read_csv('./band_user_data_selenium - Copy1.csv', low_memory=False, on_bad_lines='warn')

In [ ]:
# 중복 제거
try:
    df = df.drop_duplicates(subset=['ID'], keep='first')
except Exception as e:
    print(f"Error: {e}")
    pass

In [ ]:
# 결과 저장
df.to_csv('your_file_no_duplicates.csv', index=False)